# Project 2: Cavity number preprocessing

GOAL: develop a program to preprocess an image and get it ready to perform
the OCR of the cavity number of a plastic cap. 

The cap has an external tab at a fixed position in relation to the cavity number. The program should give as output the rectified crop containing the cavity number. 

The provided ground thruth image set contains 29 grayscale images, all with a cavity number and all presented with different rotation angle.

In [1]:
# Load necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import time

from src.utils import *
import src.config as config

%load_ext autoreload
%autoreload 2

## 1. Read images

Images are stored in a dictionary and a Dataframe, containing images classes and features, is created.

In [2]:
df = create_df(config.path)
image_collection = read_img_gs(config.path)
df.head()

,image
0,d_01.bmp
1,d_02.bmp
2,d_03.bmp
3,d_04.bmp
4,d_05.bmp


#### Select image to work on, es. 'd_01.bmp', or uncomment the following cell and process all images in batch.

Results are stored in the "res" folder, while data in df DataFrame.

In [3]:
# df = process_all(df, image_collection, smooth=True)
# df.head()

#### From here on this notebook works on a single image, for demo purposes.

Select the image under analysis through the variable "testing_image"

In [4]:
testing_image = 'd_01.bmp'
img = image_collection[testing_image].copy()

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2. Cap detection

In this section cap and external apparatus circumferences are detected through Circle Hough Transform.

The idea is to subtract both the found circumferences to the original image in order to eliminate unwanted details and focus on the search of the cap's tab.

Configuratoin parameters have been determined by experiments and qualitative assesments, because no ground truth was provided.

Circles detection is accomplished throug OpenCV HoughCircles function. Its argument are:
- image: 8-bit, single channel image;
- method: defines the method to detect circles in images. Currently, the only implemented method is cv2.HOUGH_GRADIENT, which corresponds to the Yuen et al. paper.
- dp: this parameter is the inverse ratio of the accumulator resolution to the image resolution (see Yuen et al. for more details). Essentially, the larger the dp gets, the smaller the accumulator array gets.
- minDist: Minimum distance between the center (x, y) coordinates of detected circles. If the minDist is too small, multiple circles in the same neighborhood as the original may be (falsely) detected. If the minDist is too large, then some circles may not be detected at all.
- param1: First method-specific parameter. In case of HOUGH_GRADIENT , it is the higher threshold of the two passed to the Canny edge detector (the lower one is twice smaller).
- param2: Accumulator threshold value for the cv2.HOUGH_GRADIENT method. The smaller the threshold is, the more circles will be detected (including false circles).
- minRadius: Minimum size of the radius (in pixels).
- maxRadius: Maximum size of the radius (in pixels).

In [5]:
## Cap detection
cap = single_detect_circles(img, config.cap_minDist, config.cap_param1, config.cap_param2, config.cap_minRadius, config.cap_maxRadius, print_result=False, show=True, save_img_as='p2_cap_cht.jpg')
cap_a, cap_b ,cap_r = cap

print(f'Cap center = ({cap_a},{cap_b})')
print(f'Cap radius = {cap_r}px')

Cap center = (382,288)
Cap radius = 227px


In [6]:
## Background detection
bg = single_detect_circles(img, config.bg_minDist, config.bg_param1, config.bg_param2, config.bg_minRadius, config.bg_maxRadius, print_result=False, show=True, save_img_as='p2_outer_apparat_cht.jpg')

bg_a, bg_b ,bg_r = bg
print(f'Background center = ({bg_a},{bg_b})')
print(f'Background radius = {bg_r}px')

Background center = (398,318)
Background radius = 377px


## 3. Binarization and morphology transformation

In this section the image is first binarized using Otsu's algorithm and then the two circumfercens found before are used to remove from the binary image both the cap and the external industrial apparatus visible in the images.

What is left is the cap's tab and some borders residues.

These residues are dealt by morphological opening by a $7\times 11$ structuring element that cancels out everything but the cap's tab.

In [7]:
# Image binarization
binary = cv2.threshold(img, 0, 255,	cv2.THRESH_OTSU)[1]

cv2.imshow('thresh',binary)
# cv2.imwrite(f'{config.report_img_path}Otsu.jpg',binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
masked = binary.copy()

# Draw a circular mask of the size of the cap
mask = np.zeros(binary.shape, dtype=np.uint8)
cap_mask = cv2.circle(mask, (cap_a,cap_b), cap_r, 255, -1)
cap_masked = cv2.bitwise_and(masked, masked, mask=~cap_mask)

cv2.imshow('masked',cap_masked)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# Draw a circular mask of the size of the external industrial apparatus 
bg_mask = cv2.circle(mask, (bg_a,bg_b), bg_r, 255, -1)
bg_masked = cv2.bitwise_and(cap_masked, cap_masked, mask=bg_mask)

cv2.imshow('masked',bg_masked)
# cv2.imwrite(f'{config.report_img_path}full_masked.jpg',bg_masked)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# Apply morphology open with 7x11 structuring element to cancel out cap's border residues
kernel = np.ones((7,11),np.uint8)
morph = cv2.morphologyEx(bg_masked, cv2.MORPH_OPEN, kernel)

cv2.imshow('morph',morph)
# cv2.imwrite(f'{config.report_img_path}opening.jpg',morph)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 4. Connected component detection

Now the image contains only (most of times) the binary representation of the cap's tab, so it is possible to search for connected component in order to compute tab's barycenter.

Sometimes some border's residues can survive the previous morphological transformation: when it happens they are considered as connected component but they are immediately discarded because of their form factor. Indeed, for morphological reasons, they presents a high form factor and so I can discard them by taking the connected component with the min form factor.

The form factor used in this project is defined as: $\frac{P^2}{4\pi A}$, and it is around 1 for the cap's tab.

Finally the line connecting cap's center to tab's barycenter is drawn.

In [11]:
# Search for connected components and their features.
(numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(morph, 4, cv2.CV_32S)

In [12]:
# Select the right connected components as the one with the highest form factor
selected_cc = select_cc(img, numLabels, labels, stats, centroids, show=True, verbose=True) # cup's tab

# Draw the line from the cap's center to the tab's barycenter
line_to_cc = link_cc_to_center(img, [cap_a, cap_b], centroids[selected_cc])

cv2.imshow("line_to_cc", line_to_cc)
cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] - examining component 1/2 (background)
Skipping background component...
[INFO] - examining component 2/2
Blob 1's form factor: 1516289.69


## 5. Vertical alignment and cropping

Now, knowing tab's baryncenter and cap's center, the rotation matrix $M$ that vertically aligns the tab can be computed and used. Rotation matrix is computed with respect to cap's center, otherwise it wouldn't work when the cap is off-center in the image.

After the rotation, a parametric crop is performed: the crop center is taken based on tab' barycenter-cap's center distance, in order to have scale invariance for the crop operation.

In [41]:
# Compute rotation matrix from anywhere to vertical aligned cap's tab
M, theta = get_vertical_aligned_rot_matrix([cap_a, cap_b], centroids[selected_cc])

print(f'Rotation of {theta} degrees w.r.t. a vertical oriented axis')
(h, w) = img.shape[:2]

# Rotate the img by theta degrees around the cap center
rotated = cv2.warpAffine(line_to_cc, M, (w, h))
img_rotated = cv2.warpAffine(img.copy(), M, (w, h))
cv2.imshow("Rotated by theta Degrees", rotated)
cv2.imshow("Original image rotated by theta Degrees", img_rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

Rotation of 323.0 degrees w.r.t. a vertical oriented axis


In [15]:
# Use this cell to check for rotation

check = rotated.copy()
cv2.line(check, (cap_a, cap_b), (cap_a, 0), (0,0,255), 1, cv2.LINE_AA)

cv2.imshow("Rotated by theta Degrees", check)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
# Crop cavity number
c = centroids[selected_cc]
rc = rotate_point(c, M)

crop, vertices = crop_cavity_number(img_rotated, [cap_a, cap_b], rc)

cv2.imshow("crop", crop)
cv2.imwrite("cavity_crop.jpg", crop)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
# Usato per report
# center = (int(cap_a), int(cap_b))

# rectified_img = polar_rectification(img, center, cap_r)
# cv2.imwrite(f'{config.report_img_path}polar_transf.jpg', rectified_img)
# cv2.imshow("rectified_img", rectified_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## 6. Polar rectification and crop again

Once the crop is obtained, it gets rectified and cropped again.

In order to obtain the final crop a maximal enclosed rectangle must to be find in rectified crop: this is done by the **find_enclosing_rectangle** function, that first computes rectified crop borders and then try to build all the possible rectangles until the one that stays in the crop and with max area is found.

In [26]:

x1, y1 = vertices[:2]

center = (int(cap_a-x1), int(cap_b-y1))

rectified_img = polar_rectification(crop, center, cap_r)

cv2.imshow("rectified_img", rectified_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
rectified_crop, coords = find_enclosing_rectangle(rectified_img, show=True, save=True)

if rectified_crop is not None:
    cv2.imwrite("rectified_crop.jpg", rectified_crop)
